In [1]:
import pandas as pd
import numpy as np
import pandas_gbq
import os.path
from pandas.io.json import json_normalize
from datetime import datetime
pd.options.display.max_columns = None
import altair as alt
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import random

In [2]:
# tickets = """
# WITH
#   recent_merchants AS (
#   SELECT
#     merchant_id,
#     date(merchant_created_at) onboarding
# from
#     `bharatpe-analytics-prod.bharatpe_data_platfrom.universal_merchant_data_mart`

#   WHERE

# DATE(merchant_created_at) >= DATE_SUB(CURRENT_DATE(), INTERVAL 60 DAY)

# ),

# ticket as (
# SELECT
#   t.merchant_id,
#   rm.onboarding,
#   t.type,
#   date(t.created_at) ticket_created,
#   count(t.id) as tickets

# FROM
#   `bharatpe-analytics-prod.bharatpe_analytics_data.fw_tickets` t
# inner join recent_merchants rm
# on rm.merchant_id = t.merchant_id
# WHERE
#   DATE(t.created_at) >= DATE_SUB(CURRENT_DATE(), INTERVAL 60 DAY) 
# group by 1,2,3,4),

# final as (

# select merchant_id,onboarding, type, ticket_created, tickets, 
# case when ticket_created >= onboarding and ticket_created < DATE_SUB(onboarding, INTERVAL -3 DAY) then tickets else 0 end as d3_tickets,
# case when ticket_created >= DATE_SUB(onboarding, INTERVAL -3 DAY) and ticket_created < DATE_SUB(onboarding, INTERVAL -7 DAY) then tickets else 0 end as d7_tickets,
# case when ticket_created >= DATE_SUB(onboarding, INTERVAL -7 DAY) and ticket_created < DATE_SUB(onboarding, INTERVAL -14 DAY) then tickets else 0 end as d14_tickets,
# case when ticket_created >= DATE_SUB(onboarding, INTERVAL -14 DAY) and ticket_created < DATE_SUB(onboarding, INTERVAL -21 DAY) then tickets else 0 end as d21_tickets,
# case when ticket_created >= DATE_SUB(onboarding, INTERVAL -21 DAY) and ticket_created < DATE_SUB(onboarding, INTERVAL -30 DAY) then tickets else 0 end as d30_tickets,
# case when ticket_created >= DATE_SUB(onboarding, INTERVAL -30 DAY) and ticket_created < DATE_SUB(onboarding, INTERVAL -60 DAY) then tickets else 0 end as d60_tickets
# from ticket
# )

# select type,
# sum(d3_tickets) d3,
# sum(d7_tickets) d7,
# sum(d14_tickets) d14,
# sum(d21_tickets) d21,
# sum(d30_tickets) d30,
# sum(d60_tickets) d60

# from final
# group by 1

# """

In [3]:
tickets = """
WITH
  recent_merchants AS (
  SELECT
    merchant_id,
    date(merchant_created_at) onboarding
from
    `bharatpe-analytics-prod.bharatpe_data_platfrom.universal_merchant_data_mart`

  WHERE

DATE(merchant_created_at) >= DATE_SUB(CURRENT_DATE(), INTERVAL 60 DAY)

),

ticket as (
SELECT
  t.merchant_id,
  rm.onboarding,
  t.type,
  date(t.created_at) ticket_created,
  count(t.id) as tickets

FROM
  `bharatpe-analytics-prod.bharatpe_analytics_data.fw_tickets` t
inner join recent_merchants rm
on rm.merchant_id = t.merchant_id
WHERE
  DATE(t.created_at) >= DATE_SUB(CURRENT_DATE(), INTERVAL 60 DAY) 
group by 1,2,3,4),

final as (

select merchant_id,onboarding, type, ticket_created, tickets, 
case when ticket_created >= onboarding and ticket_created < DATE_SUB(onboarding, INTERVAL -7 DAY) then tickets else 0 end as day07_tickets,
case when ticket_created >= DATE_SUB(onboarding, INTERVAL -7 DAY) and ticket_created < DATE_SUB(onboarding, INTERVAL -14 DAY) then tickets else 0 end as day14_tickets,
case when ticket_created >= DATE_SUB(onboarding, INTERVAL -14 DAY) and ticket_created < DATE_SUB(onboarding, INTERVAL -30 DAY) then tickets else 0 end as day30_tickets,
case when ticket_created >= DATE_SUB(onboarding, INTERVAL -30 DAY) and ticket_created < DATE_SUB(onboarding, INTERVAL -60 DAY) then tickets else 0 end as day60_tickets,
from ticket
)

select type,

sum(day07_tickets) day_07,
sum(day14_tickets) day_14,
sum(day30_tickets) day_30,
sum(day60_tickets) day_60

from final
group by 1

"""

In [4]:
tickets = pandas_gbq.read_gbq(tickets)

Downloading: 100%|██████████| 318/318 [00:00<00:00, 2488.64rows/s]


In [5]:
new_merchants = """
WITH
  recent_merchants AS (
  SELECT
    pt.merchant_id,
    COUNT(DISTINCT id) AS txns,
    cast(SUM(amount) as int) AS tpv
  FROM
    bharatpe-analytics-prod.payin.transactions pt
  JOIN
    `bharatpe-analytics-prod.bharatpe_data_platfrom.universal_merchant_data_mart` dm
  ON
    pt.merchant_id = dm.merchant_id
  WHERE
    status = "SUCCESS"
    AND DATE(dm.merchant_created_at) >= DATE_SUB(CURRENT_DATE(), INTERVAL 60 DAY)
    AND DATE(payment_timestamp) >= DATE_SUB(CURRENT_DATE(), INTERVAL 30 DAY)
  GROUP BY
    1
)

select * from recent_merchants

"""

In [6]:
new_merchants = pandas_gbq.read_gbq(new_merchants)

Downloading: 100%|██████████| 567610/567610 [00:10<00:00, 52801.80rows/s]


In [7]:
c1 = new_merchants['tpv'] >= 500
c2 = new_merchants['txns'] >= 15
c = c1 & c2
new_merchants['Active'] = np.where(c, 1, 0)

In [8]:
mer_type = """
with mer_type as (
SELECT
  merchant_id,
  type

FROM
  `bharatpe-analytics-prod.bharatpe_analytics_data.fw_tickets` 

WHERE
  DATE(created_at) >= DATE_SUB(CURRENT_DATE(), INTERVAL 60 DAY) 
  and merchant_id is not null
  order by merchant_id desc
)

select * from mer_type
"""

In [9]:
mer_type = pandas_gbq.read_gbq(mer_type)

Downloading: 100%|██████████| 857492/857492 [00:14<00:00, 58545.88rows/s]


In [10]:
act_dic = new_merchants.set_index('merchant_id')['Active'].to_dict()

In [11]:
mer_type['Active'] = mer_type['merchant_id'].map(act_dic)

In [12]:
# ticket = pd.melt(tickets, id_vars =['type'], value_vars =['d3','d7','d14','d21','d30','d60'],
#               var_name ='Vintage', value_name ='Value')
ticket = pd.melt(tickets, id_vars =['type'], value_vars =['day_07','day_14','day_30','day_60'],
              var_name ='Vintage', value_name ='Value')

In [13]:
top_type = list(ticket.groupby(['type'])['Value'].sum().sort_values(ascending = False)[:20].index)

In [14]:
type_act = pd.DataFrame(100*mer_type[mer_type['type'].isin(top_type)].groupby('type')['Active'].mean()).reset_index()

In [15]:
mer_type = mer_type[mer_type['merchant_id'].isin(new_merchants['merchant_id'])]

In [16]:
ticket = ticket[ticket['type'].isin(top_type)]

In [17]:
tot_types = ['Voice notification not working No sound at all - Speaker',
 'BharatPe Card Application on Hold',
 'Other Complaint',
 'Return Request - Speaker',
 'Unable to do KYC',
 'Information about BharaPe Loan',
 'Settlement Status- Block',
 'Settlement Status -Pending',
 'Info -Features and Charges of Bharatswipe (New Machine)',
 'Speaker not delivered Within TAT',
 'Book A Return',
 'Incomplete call',
 'How to Order Physical QR code',
 'New App - Features & Benefits',
 'Info - Status of Application',
 'Status of Withdrawal',
 'Status of Withdrawal - Pending / Failed',
 'Speaker Delivered But Showing Ordered on CRM',
 'Language Barrier',
 'Interest amount calculation',
 'Information on Settings/Usage - Speaker Active',
 'Need Information about Charges applied',
 'Not able to see any transactions in the app',
 'Information on withdrawal',
 'Enquiry about another number',
 'Transfer Success but not reflecting in Bank',
 'Information about Eligibility Criteria',
 'Set Up Information',
 'Reason for Ineligible',
 'Other',
 'Blank Call',
 'Speaker not delivered Beyond TAT',
 'Speaker - Application/Profile Download failed',
 'Reason for Rejection/ Expiry',
 'Overall features of BharatPe Speaker']

In [18]:
color = ['#D1FBEF',
 '#75AD66',
 '#5F11D9',
 '#119DBA',
 '#96C35F',
 '#46F2ED',
 '#E28D81',
 '#16D3B4',
 '#246217',
 '#BCCB51',
 '#260B10',
 '#D38AF6',
 '#1A5B48',
 '#D795B6',
 '#CE8BD2',
 '#70FC9B',
 '#C3A378',
 '#97133A',
 '#EDBB8C',
 '#E63B55',
 '#2EE284',
 '#DE5136',
 '#71407C',
 '#DFA6ED',
 '#4DF874',
 '#6EF4BC',
 '#6FF85C',
 '#CACBE7',
 '#90BD1A',
 '#62E9A7',
 '#681FB4',
 '#81FBE7',
 '#5650BF',
 '#4CBD96',
 '#D79E45']

In [19]:
color_scale = alt.Scale(
    domain=tot_types,
    range=color,
)

In [31]:
def chart(fil):
    # fil = "d3"
    alt.themes.enable('fivethirtyeight')
    d3_type = list(ticket[ticket['Vintage'] == fil].groupby('type')['Value'].sum().sort_values(ascending = False).index[:20])
    base = alt.Chart(ticket[ticket['Vintage'] == fil], title = f"Top Issues till Day {fil[4:]}").encode(y=alt.Y('type:N',axis=alt.Axis(title=""), sort = "-x",stack='normalize'),
                                 tooltip = ['type'], color=alt.Color("type:N", scale=color_scale, legend = None)).transform_window(
        rank='rank(Value)',
        sort=[alt.SortField('Value', order='descending')]
    ).transform_filter(
        (alt.datum.rank < 20)
    )

    bar = base.mark_bar(color = 'lightgrey').encode(x=alt.X('Value:Q',axis=alt.Axis(title="Issue Count")))

    txt_bar = bar.mark_text(align='left',
            baseline='middle',angle = 0
    ).encode(
        x=alt.X('Value:Q',axis=alt.Axis(title="",labels=False,ticks=True)),
        text=alt.Text('Value:Q',format='1.0d')
    )

    d3 = (bar + txt_bar).properties(height = 500, width = 45)
    return d3, d3_type

In [32]:
# d3 = chart('d3')[0]
d7 = chart('day_07')[0]
d14 = chart('day_14')[0]
# d21 = chart('d21')[0]
d30 = chart('day_30')[0]
d60 = chart('day_60')[0]

In [33]:
# (d3 | d7 | d14 | d21 |d30 |d60)
(d7 | d14 |d30 |d60)

alt.HConcatChart(...)

In [23]:
# tot_types = list(set(chart('d3')[1]).union(set(chart('d7')[1])).union(set(chart('d14')[1])).union(set(chart('d21')[1])).union(set(chart('d30')[1])).union(set(chart('d60')[1])))

In [24]:
# import random

# number_of_colors = len(tot_types)

# color = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
#              for i in range(number_of_colors)]

In [25]:
ticket.head()

,type,Vintage,Value
1,Settlement Status- Block,day_07,327
2,New App - Features & Benefits,day_07,8280
3,BharatPe Card Application on Hold,day_07,652
7,Blank Call,day_07,663
10,Reason for Ineligible,day_07,1075


In [26]:
alt.themes.enable('fivethirtyeight')
bump = alt.Chart(ticket).mark_line(point=True).encode(
 x = alt.X("Vintage:O", title = 'Vintage in Days'),
 y = alt.Y('rank:O', title = 'Ranking'),
 color = alt.Color('type:N', legend=None)
).transform_window(
 rank = "rank()",
 sort=[alt.SortField('Value', order="descending")],
 groupby=["Vintage"]
).properties(
 title = 'Rank Plot for Tickets Raised Vs Merchant Vintage',
    width = 800, height = 600
)

In [27]:
labels = alt.Chart(ticket).mark_text(align='left', dx=12).encode(
 x = alt.X("Vintage:O", title = 'Vintage in Days', aggregate='max'),
 y = alt.Y('rank:O', title = 'Ranking', aggregate={'argmax': 'Vintage'}),
 text = alt.Text('type:N'),
 color = alt.Color('type:N', legend=None)
).transform_window(
 rank = "rank()",
 sort=[alt.SortField('Value', order="descending")],
 groupby=["Vintage"]
)

In [28]:
(bump + labels)

alt.LayerChart(...)

In [30]:
bar = alt.Chart(type_act).mark_bar().encode(
x = alt.X('type:N',sort = "-y"),
    y = 'Active:Q',
    tooltip = ['type:N']
)

txt = alt.Chart(type_act).mark_text(align='left',
            baseline='middle',angle = 0, dy = -6
    ).encode(
x = alt.X('type:N',sort = "-y"),
    y = 'Active:Q',
    text=alt.Text('Active:Q',format='1.0d')
)
(bar + txt).properties(title = "Activation on Issue type basis", height = 400, width = 700)

alt.LayerChart(...)